In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')

f = x*x*y + y + 2

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)
print(result)

42


In [3]:
sess.close()

In [4]:
with tf.Session() as sess: 
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [5]:
print(result)

42


In [6]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    
print(result)

42


In [7]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()

print(result)
sess.close()

42


In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [9]:
graph = tf.Graph()

with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [10]:
x2.graph is tf.get_default_graph()

False

In [11]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [12]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [13]:
# linear regression via normal equation
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
housing_data_scaled = scaler.fit_transform(housing.data.astype(np.float32))
housing_data_scaled_plus_bias = np.c_[np.ones((m, 1)), housing_data_scaled]

# Linear reg via gradient descent
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0))
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    print(best_theta)


Epoch 0 MSE = 5.7111
Epoch 100 MSE = 0.645484
Epoch 200 MSE = 0.577376
Epoch 300 MSE = 0.564108
Epoch 400 MSE = 0.554846
Epoch 500 MSE = 0.547876
Epoch 600 MSE = 0.542593
Epoch 700 MSE = 0.538568
Epoch 800 MSE = 0.535486
Epoch 900 MSE = 0.533116
[[ 2.06855369]
 [ 0.87773645]
 [ 0.14678799]
 [-0.32087246]
 [ 0.33582199]
 [ 0.00507181]
 [-0.04272393]
 [-0.65032727]
 [-0.62486237]]


In [15]:
# Linear reg via gradient descent with autodiff
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0))
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs): 
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    
    print(best_theta)
        

Epoch 0 MSE = 12.1629
Epoch 100 MSE = 0.842751
Epoch 200 MSE = 0.648684
Epoch 300 MSE = 0.614975
Epoch 400 MSE = 0.592068
Epoch 500 MSE = 0.575183
Epoch 600 MSE = 0.562672
Epoch 700 MSE = 0.55337
Epoch 800 MSE = 0.546429
Epoch 900 MSE = 0.54123
[[ 2.06855345]
 [ 0.8676033 ]
 [ 0.15876412]
 [-0.27577448]
 [ 0.28711629]
 [ 0.00956009]
 [-0.04352097]
 [-0.57254004]
 [-0.54464036]]


In [16]:
# Linear reg via gradient descent with optimizer
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0))
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs): 
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    
    print(best_theta)

Epoch 0 MSE = 12.1556
Epoch 100 MSE = 0.783033
Epoch 200 MSE = 0.599959
Epoch 300 MSE = 0.580547
Epoch 400 MSE = 0.568047
Epoch 500 MSE = 0.558545
Epoch 600 MSE = 0.551246
Epoch 700 MSE = 0.545608
Epoch 800 MSE = 0.54123
Epoch 900 MSE = 0.537813
[[ 2.06855345]
 [ 0.90010285]
 [ 0.1525289 ]
 [-0.36023676]
 [ 0.36710042]
 [ 0.00687008]
 [-0.04368649]
 [-0.58747631]
 [-0.5644747 ]]


In [25]:
# Linear reg via gradient descent with momentum optimizer
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0))
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs): 
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/my_model.ckpt')
    
    print(best_theta)

Epoch 0 MSE = 11.4892
Epoch 100 MSE = 0.526436
Epoch 200 MSE = 0.52439
Epoch 300 MSE = 0.524327
Epoch 400 MSE = 0.524321
Epoch 500 MSE = 0.524321
Epoch 600 MSE = 0.524321
Epoch 700 MSE = 0.524321
Epoch 800 MSE = 0.524321
Epoch 900 MSE = 0.52432
[[ 2.06855989]
 [ 0.82962185]
 [ 0.1187521 ]
 [-0.2655316 ]
 [ 0.30570012]
 [-0.00450287]
 [-0.03932637]
 [-0.89988011]
 [-0.87053579]]


In [26]:
# Linear reg via mini batch gradient descent
n_epochs = 1000
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

# use placeholders for the batches
X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)
    X_batch = housing_data_scaled_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0))
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs): 
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch })
    best_theta = theta.eval()
    
    print(best_theta)

[[ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]]


array([61, 60, 32, 25, 36, 59, 79, 65, 88,  9, 72, 48, 52, 61, 58, 32, 42,
       71, 84, 71, 30, 49, 96, 93, 80, 96, 58, 41, 83, 86, 60, 56, 67, 47,
       83, 65, 13, 98, 97, 60, 20, 16, 81, 65, 26, 64, 56, 43, 99, 46, 28,
        4, 16, 32, 38, 35, 13, 31, 21, 51, 44, 81, 29,  7, 39, 85, 32, 51,
       21, 21, 50, 53, 55, 42, 60, 20, 48, 65, 81, 50, 58,  7, 65,  6, 60,
       58, 99, 78, 56, 77, 57,  6, 95,  8, 30, 68, 59, 79, 48, 29, 59, 23,
       59, 83, 30, 39, 13, 70,  0, 45,  7, 76, 10, 46, 88, 82, 95, 68, 35,
       77, 62,  1, 28, 66, 20, 17, 89, 60, 44, 89, 23, 53, 41, 86, 54, 59,
       95, 84, 61, 80,  9, 71,  8, 14, 62, 87, 10, 74, 81, 47, 25, 35, 16,
       30, 91, 61, 74, 28, 94, 91, 15, 71, 77, 43, 86, 49, 40,  0, 88, 89,
       68, 74, 71, 17, 72, 73, 32, 79, 71, 54, 86, 25, 51, 45, 15, 98, 36,
       43,  0,  3, 41, 90, 14, 50, 80, 97, 70, 41, 42, 11, 44, 83, 49, 73,
       99, 12,  8, 44, 74, 68, 76, 46, 97,  0, 68, 58, 19, 74,  6, 17, 60,
       76,  2, 34, 92, 23

In [28]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = 'tf_logs'
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0))
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs): 
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        if epoch % 10 == 0:
            summary_str = mse_summary.eval()
            step = epoch 
            file_writer.add_summary(summary_str, step)
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/my_model.ckpt')
    
    print(best_theta)
    file_writer.close()

Epoch 0 MSE = 7.33141
Epoch 100 MSE = 0.530471
Epoch 200 MSE = 0.524635
Epoch 300 MSE = 0.524354
Epoch 400 MSE = 0.524325
Epoch 500 MSE = 0.524322
Epoch 600 MSE = 0.524321
Epoch 700 MSE = 0.524321
Epoch 800 MSE = 0.52432
Epoch 900 MSE = 0.524321
[[ 2.06855965]
 [ 0.82961375]
 [ 0.11875057]
 [-0.26551634]
 [ 0.30568749]
 [-0.0045033 ]
 [-0.03932607]
 [-0.89989847]
 [-0.87055314]]
